# Rekurencyjne Sieci Neuronowe (RNN)

### Importy i Utilsy  (odpalić i schować )

In [1]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaci kodowania "one-hot". Inaczej mówiąc, każde nazwisko jest binarną macierzą rozmiaru `len(name)` $\times$ `n_letters`. 

Dodatkowo, ponieważ ten dataset jest mocno niezbalansowany, użyjemy specjalnego samplera do losowania przykładów treningowych, tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [2]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2021-01-20 16:16:13--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.32.204.65, 13.32.204.34, 13.32.204.49, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.32.204.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>]   2.75M  --.-KB/s    in 0.01s   

2021-01-20 16:16:13 (209 MB/s) - ‘data.zip.1’ saved [2882130/2882130]

Archive:  data.zip
replace data/eng-fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/eng-fra.txt        
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inf

In [3]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [4]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    break

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

x.shape: torch.Size([1, 2, 52])
name: Ta
y: vietnamese


## Zadanie 1. (2 pkt.)

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci możemy mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca stan ukryty na wyjście.
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".
* Zalecane jest użycie aktywacji na warstwie liczącej reprezentacje `hidden` tak, aby nie "eksplodowała", np. `tanh`.


In [5]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.hidden_to_output = torch.nn.Linear(input_size + hidden_size, output_size)
    
        self.softmax = torch.nn.LogSoftmax(dim=1)
        self.tanh = torch.nn.Tanh()

    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """
        
        combined = torch.cat([input, hidden], dim=1) 
        hidden = self.input_to_hidden(combined)
        hidden = self.tanh(hidden)

        output = self.hidden_to_output(combined)
        output = self.softmax(output)

        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda()

### Pętla uczenia

In [6]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class).cuda()
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

# we will train for only a single epoch 
epochs = 1

# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn

        seq_len = x.shape[1]

        for j in range(seq_len):
          output, hidden = rnn(x[:,j], hidden)
            
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        x = x.cuda()
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
 
        for i in range(seq_len):
          output, hidden = rnn(x[:,i], hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.912
Epoch: 0 Progress:  6% Loss: 2.848
Epoch: 0 Progress: 11% Loss: 2.739
Epoch: 0 Progress: 17% Loss: 2.495
Epoch: 0 Progress: 22% Loss: 2.344
Epoch: 0 Progress: 28% Loss: 2.196
Epoch: 0 Progress: 33% Loss: 2.151
Epoch: 0 Progress: 39% Loss: 2.061
Epoch: 0 Progress: 44% Loss: 1.995
Epoch: 0 Progress: 50% Loss: 1.934
Epoch: 0 Progress: 55% Loss: 1.903
Epoch: 0 Progress: 61% Loss: 1.849
Epoch: 0 Progress: 66% Loss: 1.831
Epoch: 0 Progress: 72% Loss: 1.839
Epoch: 0 Progress: 77% Loss: 1.722
Epoch: 0 Progress: 83% Loss: 1.705
Epoch: 0 Progress: 89% Loss: 1.683
Epoch: 0 Progress: 94% Loss: 1.632
Epoch: 0 Progress: 100% Loss: 1.705
Final F1 score: 0.19


## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [7]:
def predict(name: str, rnn: RNN):
    """Prints the name and model's top 3 predictions with scores"""
    x = line_to_tensor(name).view(1, len(name), -1)
    x = x.cuda()

    hidden = rnn.init_hidden(x.shape[0])

    for i in range(x.shape[1]):
      output, hidden = rnn(x[:,i], hidden)
      
    best_pred = torch.topk(output, k=3, dim=1)[1]
    for i, pred in enumerate(best_pred.data[0]):
      print(f"Prediction {i+1}: {label_to_idx[pred.item()]}")


In [8]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
Prediction 1: japanese
Prediction 2: portuguese
Prediction 3: italian
Jackson
Prediction 1: scottish
Prediction 2: english
Prediction 3: dutch
Schmidhuber
Prediction 1: german
Prediction 2: dutch
Prediction 3: russian
Hinton
Prediction 1: english
Prediction 2: scottish
Prediction 3: russian
Kowalski
Prediction 1: polish
Prediction 2: russian
Prediction 3: japanese


## Zadanie 3 (4 pkt.)
Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna loginka LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [9]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task

        self.U_f = torch.nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_f = torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = torch.nn.Parameter(torch.Tensor(hidden_size))
    
        self.U_i = torch.nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_i = torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = torch.nn.Parameter(torch.Tensor(hidden_size))
        
        self.U_c = torch.nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_c = torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_c = torch.nn.Parameter(torch.Tensor(hidden_size))
        
        self.U_o = torch.nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_o = torch.nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = torch.nn.Parameter(torch.Tensor(hidden_size))

        self.sigmoid = torch.nn.Sigmoid()
        self.tanh = torch.nn.Tanh()



    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states
        
        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 

        f_t = self.sigmoid(input @ self.U_f + hidden @ self.V_f + self.b_f)
        i_t = self.sigmoid(input @ self.U_i + hidden @ self.V_i + self.b_i)
        g_t = self.tanh(input @ self.U_c + hidden @ self.V_c + self.b_c)
        o_t = self.sigmoid(input @ self.U_o + hidden @ self.V_o + self.b_o)

        cell = f_t * cell + i_t * g_t 
        hidden = o_t * self.tanh(cell)

        return hidden, cell

### Klasa modelu LSTM

In [10]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        """
        
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        time_steps = input.shape[1]


        for i in range(time_steps):
          hidden, cell = self.cell(input[:,i], (hidden, cell))
          hiddens.append(hidden.unsqueeze(0))
          cells.append(cell.unsqueeze(0))

        hiddens = torch.cat(hiddens, dim=0)
        cells = torch.cat(cells, dim=0)


        return hiddens, cells
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda(), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True).cuda())

### Pętla uczenia

In [11]:
from itertools import chain

torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128).cuda()
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx)).cuda()

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.001) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the sequence length here
        # don't forget about the classifier!

        output, _ = lstm(x)

        pred = clf(output[-1])
        loss = cross_entropy(pred, y)
        loss.backward()
        optimizer.step()                                
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        x = x.cuda()
        ys.append(y.numpy())
        
        output, _ = lstm(x)
        output = clf(output[-1])
        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.845
Epoch: 0 Progress:  6% Loss: 2.715
Epoch: 0 Progress: 11% Loss: 2.188
Epoch: 0 Progress: 17% Loss: 1.796
Epoch: 0 Progress: 22% Loss: 1.769
Epoch: 0 Progress: 28% Loss: 1.619
Epoch: 0 Progress: 33% Loss: 1.582
Epoch: 0 Progress: 39% Loss: 1.466
Epoch: 0 Progress: 44% Loss: 1.417
Epoch: 0 Progress: 50% Loss: 1.348
Epoch: 0 Progress: 55% Loss: 1.309
Epoch: 0 Progress: 61% Loss: 1.247
Epoch: 0 Progress: 66% Loss: 1.248
Epoch: 0 Progress: 72% Loss: 1.188
Epoch: 0 Progress: 77% Loss: 1.139
Epoch: 0 Progress: 83% Loss: 1.075
Epoch: 0 Progress: 89% Loss: 1.042
Epoch: 0 Progress: 94% Loss: 0.928
Epoch: 0 Progress: 100% Loss: 0.983
Final F1 score: 0.23


## Zadanie 4. (0.5 pkt.)
Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [12]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    """Prints the name and model's top 3 predictions with scores"""
    x = line_to_tensor(name).view(1, len(name), -1)
    x = x.cuda()

    output, _ = lstm(x)
    out_clf = clf(output[-1])
    
    best_pred = torch.topk(out_clf, k=3, dim=1)[1]
    for i, pred in enumerate(best_pred.data[0]):
      print(f"Prediction {i+1}: {label_to_idx[pred.item()]}")


In [13]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
Prediction 1: arabic
Prediction 2: japanese
Prediction 3: italian
Jackson
Prediction 1: scottish
Prediction 2: english
Prediction 3: dutch
Schmidhuber
Prediction 1: german
Prediction 2: english
Prediction 3: dutch
Hinton
Prediction 1: english
Prediction 2: scottish
Prediction 3: german
Kowalski
Prediction 1: polish
Prediction 2: czech
Prediction 3: japanese
